In [1]:
# 透视表和交叉表
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [2]:
tips = pd.read_csv('../data/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [3]:
tips.pivot_table(index=['time', 'smoker']) # 使用time和smoker作为index创建透视表。书里的rows=估计是旧代码，会报错。

size       tip   tip_pct  total_bill
time   smoker                                          
Dinner No      2.735849  3.126887  0.158653   20.095660
       Yes     2.471429  3.066000  0.160828   21.859429
Lunch  No      2.511111  2.673778  0.160920   17.050889
       Yes     2.217391  2.834348  0.170404   17.399130

In [4]:
# 书上代码和实际差太多了。。。看了若干参考文档后调整如下：
# values代表要保留的列
# index代表把哪些列变成行索引
# columns表示针对哪些列进行汇总统计
# margins=True表示做分项小计
# 另外还有个变量margins_name,默认为All。现在指定Smoker列。
tips.pivot_table(values=['tip_pct', 'size'],
                 index=['time', 'day'],
                 columns=['smoker'],
                 margins=True,
                 margins_name='Smoker')

size                       tip_pct                    
smoker             No       Yes    Smoker        No       Yes    Smoker
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
Smoker       2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [5]:
tips.pivot_table(values='tip_pct', # 被统计对象
                 index=['time', 'smoker'],
                 columns='day',
                 aggfunc=np.mean,
                 margins=True) 

day                 Fri       Sat       Sun      Thur       All
time   smoker                                                  
Dinner No      0.139622  0.158048  0.160113  0.159744  0.158653
       Yes     0.165347  0.147906  0.187250       NaN  0.160828
Lunch  No      0.187735       NaN       NaN  0.160311  0.160920
       Yes     0.188937       NaN       NaN  0.163863  0.170404
All            0.169913  0.153152  0.166897  0.161276  0.160803

In [6]:
tips.pivot_table('size',
                 index=['time', 'smoker'],
                 columns='day',
                 aggfunc='sum',
                 fill_value=0) # 缺失值用0填充

day            Fri  Sat  Sun  Thur
time   smoker                     
Dinner No        6  115  167     2
       Yes      20  104   49     0
Lunch  No        3    0    0   110
       Yes      11    0    0    40

In [7]:
# pivot_table的参数：
# values：待聚合的列的名称，默认聚合所有数值列。
# index：用于分组的列名或其他分组键，出现在结果透视表的行。
# columns：用于分组的列名或其他分组键，出现在结果透视表的列。
# aggfunc：聚合函数或函数列表，默认为mean。可以是任何对groupby有效的函数。
# margins：添加行/列小计和总结，默认为False。
# margins_name：需要添加总结的行/列名称，默认为All（如果margins为True）。

In [8]:
# 交叉表：crosstab

In [9]:
# 书上也没给出数据怎么来的，只能自己构造了。
sample = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
gender = ['F', 'M', 'F', 'M', 'M', 'M', 'F', 'F', 'M', 'F']
handedness = ['R', 'L', 'R', 'R', 'L', 'R', 'R', 'L', 'R', 'R']
data = pd.DataFrame({'Sample': sample, 'Gender': gender, 'Handedness': handedness},
                    columns=['Sample', 'Gender', 'Handedness'])
data

,Sample,Gender,Handedness
0,1,F,R
1,2,M,L
2,3,F,R
3,4,M,R
4,5,M,L
5,6,M,R
6,7,F,R
7,8,F,L
8,9,M,R
9,10,F,R


In [10]:
pd.crosstab(data.Gender, data.Handedness, margins=True) # 根据性别和手向生成交叉表，分别汇总统计数量。

Handedness,L,R,All
Gender,,,
F,1,4,5
M,2,3,5
All,3,7,10


In [11]:
# 行索引是time/day
# 针对smoke做交叉表，列名是smoker，因为margins=True，所以统计all。
ct = pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)
ct

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [12]:
ct.columns

Index(['No', 'Yes', 'All'], dtype='object', name='smoker')

In [13]:
ct.index

MultiIndex(levels=[['All', 'Dinner', 'Lunch'], ['', 'Fri', 'Sat', 'Sun', 'Thur']],
           labels=[[1, 1, 1, 1, 2, 2, 0], [1, 2, 3, 4, 1, 4, 0]],
           names=['time', 'day'])